In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("table.csv")

In [ ]:

new_df = df[['WP\ncode','Active\nusers']].drop_duplicates().reset_index(drop=True)

# Create an 'id' field starting at 0
new_df['id'] = range(len(new_df))

# Ensure 'en' and 'simple' have the same id value
en_index = new_df[new_df['WP\ncode'] == 'en'].index
simple_index = new_df[new_df['WP\ncode'] == 'simple'].index

if not en_index.empty and not simple_index.empty:
    en_value = new_df.loc[en_index, 'id'].values[0]
    new_df.loc[simple_index, 'id'] = en_value
    
    # Adjust the ids for rows after 'simple'
    rows_to_adjust = new_df.index > simple_index[0]
    new_df.loc[rows_to_adjust, 'id'] -= 1

In [ ]:
new_df.sort_values("id")

In [ ]:
new_df.loc[new_df['id'] < 128]['Active\nusers'].str.replace(',', '').astype(int).sum()

In [ ]:
from sqlalchemy import SmallInteger, String
# Insert the new DataFrame into the database
new_df.to_sql('wikipedia_lang_code', engine, if_exists='replace', index=False, 
              dtype={'id': SmallInteger, 'WP\ncode': String})

print("Table 'wikipedia_lang_code' has been created/updated in the database.")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import concurrent.futures
import psycopg2
from psycopg2.extras import execute_values

# Connect to the database
connection_string = "" # SQL connection string, e.g. 'postgresql://muchane@localhost/interp'
db_name = "" # SQL database name, e.g. 'interp'
username = "" # DB username, e.g. 'muchane'
url = "" # DB url/IP address, e.g. 127.0.0.1

engine = create_engine('')

# Load the wikipedia_lang_code table into a DataFrame
lang_codes_df = new_df

# Create a dictionary for faster lookups
lang_id_dict = dict(zip(lang_codes_df['WP\ncode'], lang_codes_df['id']))

# Function to process a chunk of rows
def process_chunk(chunk):
    conn = psycopg2.connect(f"dbname={db_name} user={username} host={url}")
    cur = conn.cursor()
    
    # Prepare data for updating
    data = [(lang_id_dict.get(lang, None), id) for lang, id in zip(chunk['lang'], chunk['id'])]
    
    # Update the chunk
    execute_values(cur, 
                   "UPDATE wikipedia_text SET lang_id = data.lang_id FROM (VALUES %s) AS data(lang_id, id) WHERE wikipedia_text.id = data.id",
                   data)
    
    conn.commit()
    cur.close()
    conn.close()

# Add lang_id column if it doesn't exist
with engine.connect() as connection:
    connection.execute(text("ALTER TABLE wikipedia_text ADD COLUMN IF NOT EXISTS lang_id SMALLINT"))

# Process the wikipedia_text table in chunks
chunksize = 10000  # Adjust this based on your system's capabilities
with concurrent.futures.ThreadPoolExecutor(max_workers=64) as executor:  # Adjust max_workers as needed
    for chunk in pd.read_sql("SELECT id, lang FROM wikipedia_text", engine, chunksize=chunksize):
        executor.submit(process_chunk, chunk)

print("Processing complete. The 'lang_id' column has been added and populated.")